In [1]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

In [2]:
#Directory per Framar, Jimmy, Leo
dir=r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\Spotify_Million\data'

In [4]:
# Initialize data structures for deduplication
decoder = {}

# Helper function to get or create IDs for deduplication
def get_or_create_id(mapping, value):
    if value not in mapping:
        mapping[value] = len(mapping) + 1
    return mapping[value]

# Directory containing JSON files

# Initialize an empty list for storing playlists as rows
playlists_data = []

for filename in tqdm(os.listdir(dir)):
    if filename.endswith(".json"):
        dir_file = os.path.join(dir, filename)
        try:
            with open(dir_file, "r") as json_file:
                data = json.load(json_file)

                for playlist in data.get("playlists", []):
                    playlist_tracks = []

                    for track in playlist.get("tracks", []):
                        playlist_tracks.append(get_or_create_id(decoder, (track.get("track_uri"))))

                    # Add the playlist to the dataframe structure
                    playlists_data.append({
                        "user_id": playlist.get("pid"),
                        "name": playlist.get("name"),
                        "tracks": playlist_tracks
                    })
                    del playlist_tracks

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in file {filename}: {e}")
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")

with open('decoder.json', 'w') as decoder_file:
    json.dump(decoder, decoder_file, indent=4)

del decoder

# Convert playlists data to a Pandas DataFrame
playlists_df = pd.DataFrame(playlists_data)

del playlists_data

# Save the DataFrame to a file
output_dir = r'C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System'
os.makedirs(output_dir, exist_ok=True)

# Save as CSV instead of Parquet
output_file = os.path.join(output_dir, "playlists_trackId.csv")
playlists_df.to_csv(output_file, index=False)


print(f"Processing completed. DataFrame saved to {output_file}.")


100%|██████████| 1000/1000 [05:37<00:00,  2.96it/s]


Processing completed. DataFrame saved to C:\Users\frmar\OneDrive\Desktop\GitHub\Music-Recommender-System\playlists_trackId.csv.


In [7]:
playlists_df

,user_id,name,tracks
0,0,Throwbacks,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,1,Awesome Playlist,"[52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 6..."
2,2,korean,"[91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101,..."
3,3,mat,"[155, 156, 157, 158, 159, 160, 161, 162, 163, ..."
4,4,90s,"[281, 282, 283, 284, 285, 286, 287, 288, 289, ..."
...,...,...,...
999995,999995,Praise,"[116671, 5783, 48788, 5784, 91283, 17696, 5046..."
999996,999996,Worship,"[2262285, 2262286, 2262287, 2262288, 2262289, ..."
999997,999997,Sex,"[39643, 41143, 5434, 22056, 3860, 1043, 9152, ..."
999998,999998,✝️,"[5768, 5872, 14504, 3936, 76224, 5838, 5838, 2..."
